In [4]:
#https://depart.moe.edu.tw/ED4500/News.aspx?n=5A930C32CC6C3818&sms=91B3AAE8C6388B96
import requests
res=requests.get("http://stats.moe.gov.tw/files/school/106/e1_new.csv")

In [5]:
text=res.text.replace("\r","")

In [6]:
cols=text.split("\n")[0].split(",")

datas=[]
for i in text.split("\n")[1:]:
    data={}
    for k,v in zip(cols,i.split(",")):
        data[k]=v
        datas.append(data)

In [7]:
res=requests.get("http://stats.moe.gov.tw/files/school/106/j1_new.csv")
text=res.text.replace("\r","")
cols=text.split("\n")[0].split(",")
for i in text.split("\n")[1:]:
    data={}
    for k,v in zip(cols,i.split(",")):
        data[k]=v
        datas.append(data)

In [8]:
datas=[i for i in datas if '學校名稱' in i and i['學校名稱']!=""]

In [ ]:
import re
for dien in datas:
    dien['name']=dien.get('學校名稱',"")
    dien['address']=dien.get('地址',"")
    try:
        dien["bigadd"] = re.findall("(\w\w[市|縣])(\w\w?\w?[區|市|鎮|鄉])",dien['address'])[0][0]
        dien["smalladd"] = re.findall("(\w\w[市|縣])(\w\w?\w?[區|市|鎮|鄉])",dien['address'])[0][1]
    except:
        dien["bigadd"] = ""
        dien["smalladd"] = ""

In [ ]:
import geocoder
import time
for i in range(2):
    for i in datas:
        try:
            if 'lat' not in i:
                i['lat'],i['lng']=geocoder.arcgis(i['address'].split("]")[1].split("號")[0].split("-")[0].split("、")[0].split("~")[0]).latlng
                print(i['lat'],i['lng'])
                time.sleep(2)
        except:
            print(i['address'],"no latlng")
            pass

25.007121985284826 121.5216634926424
25.134297004154938 121.45060276836534
25.015497295602646 121.51196935512722
25.01452264640201 121.51733658811821
24.838647736337887 121.52864925284206
24.939104342777945 121.3729393878758
25.011331268271814 121.45888662747893
25.01933914377629 121.45340398819901
25.025772393848403 121.46238214628643
25.018269338334548 121.4739455856367
25.02403080532276 121.47400209234789
25.003307776299827 121.4625456869409
25.009596964193 121.47101076504711
25.032255275059075 121.47430485968616
25.02972472577764 121.47758134205678
24.989687014715187 121.43064252233259
25.022542065497174 121.45837479476594
24.9895753301635 121.41876597999307
24.99663587795422 121.4255997298334
24.981916216429493 121.41687811676918
25.001695687200677 121.41549293989854
24.97557846266379 121.40170064807545
24.969641119828275 121.39337597603435
24.95618130775035 121.38957370454162
24.950841922874734 121.34592701053161
24.94071816071317 121.33770436788048
24.972852205118514 121.3481238

In [ ]:
from pymongo import MongoClient,UpdateOne
# client=MongoClient('192.168.1.113',27017,username='j122085',password="850605")
client=MongoClient('localhost',27017,username='j122085',password="850605")
colle=client.rawData.primarySchool
ids=[i['name'] for i in datas]
operation=[UpdateOne({"_id":idn},{"$set":data},upsert=True) for idn,data in zip(ids,datas)]
colle.bulk_write(operation)
client.close()